In [4]:
%matplotlib qt 
import matplotlib.pyplot as plt
import numpy as np
import hyperspy.api as hs

## Test Signal2D 

In [8]:
from inline_holo import ModifiedImage as MI
from scipy.misc import face
data = np.moveaxis(face(), -1, 0) / 2**8
Nz, Ny, Nx = data.shape
#imgdata = imgdata * np.exp(1j*np.random.rand(Nx, Ny))
dx = 0.5
dy = 0.5
dz = 0.5
dict_x = {'size':Nx, 'name':'X', 'units':'m', 'scale':dx, 'offset':0.}
dict_y = {'size':Ny, 'name':'Y', 'units':'m', 'scale':dy, 'offset':0.}
dict_z = {'size':Nz, 'name':'Z', 'units':'m', 'scale':dz, 'offset':0.}
img = MI(data, axes=[dict_z, dict_y, dict_x])
img.metadata.General.title = 'Racoon faces'
# Add also an evil non-lineal axis
img.axes_manager[-1].axis = np.arange(Nz)**3.
# apply pad and remove it
#img = MI(np.random.rand(3, 800, 600))
cvals = hs.signals.BaseSignal([1., 0.5, 0.0])
img_pad = img.set_pad((50, 50.), 'constant', constant_values=cvals.T)
img_bis = img_pad.remove_pad()

assert np.allclose(
     img.data.shape, img_bis.data.shape), 'Shape was not preserved!'

assert np.allclose(
    img.axes_manager[0].axis, img_pad.axes_manager[0].axis), 'Pad removed the axis of evil!'

assert np.allclose(
    img.axes_manager[0].axis, img_bis.axes_manager[0].axis), 'Unpad removed the axis of evil!'

assert np.allclose(
    img_bis.data, img.data), 'Something went wrong, unpad(pad) != original'

In [9]:
hs.plot.plot_images([img, img_pad, img_bis])

## Test with Signal1D

In [14]:
from scipy.misc import face
from inline_holo import ModifiedSignal as MS
data = np.random.rand(5, 5, 512) - 0.5
Nx, Ny, Nz = data.shape
dx = 0.5
dy = 0.25
dz = 0.01
dict_x = {'size':Nx, 'name':'X', 'units':'m', 'scale':dx, 'offset':0}
dict_y = {'size':Ny, 'name':'Y', 'units':'m', 'scale':dy, 'offset':0}
dict_z = {'size':Nz, 'name':'Z', 'units':'nm', 'scale':dz, 'offset':0}
spc = hs.signals.Signal1D(data, axes=[dict_x, dict_y, dict_z])
spc = MS(spc)
spc.metadata.General.title = 'Noisy'

# apply pad and remove it
spc = hs.signals.Signal1D(np.random.rand(5, 5, 512))
spc = MS(spc)
spc_pad = spc.set_pad(((50,100),), 'constant', constant_values=0)
spc_bis = spc_pad.remove_pad()

assert np.allclose(
     spc.data.shape, spc_bis.data.shape), 'Shape was not preserved!'

assert np.allclose(
    spc.axes_manager[0].axis, spc_pad.axes_manager[0].axis), 'Pad removed the axis of evil!'

assert np.allclose(
    spc.axes_manager[0].axis, spc_bis.axes_manager[0].axis), 'Unpad removed the axis of evil!'

assert np.allclose(
    spc_bis.data, spc.data), 'Something went wrong, unpad(pad) != original'

In [15]:
spc_pad.plot()

In [16]:
spc

<ModifiedSignal, title: , dimensions: (5, 5|512)>

In [9]:
spc_pad

<ModifiedSignal, title: Noisy, dimensions: (5, 5|662)>

## Weird case

In [28]:
spc = MS(np.random.rand(5, 5, 512, 512))
spc_pad = spc.set_pad((10, 10, 0, 10), 'constant', constant_values=0)
spc_bis = spc_pad.remove_pad()

In [29]:
spc, spc_pad, spc_bis

(<ModifiedSignal, title: , dimensions: (|512, 512, 5, 5)>,
 <ModifiedSignal, title: , dimensions: (|532, 532, 5, 25)>,
 <ModifiedSignal, title: , dimensions: (|512, 512, 5, 5)>)